## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
import pandas as pd
import tensorflow as tf
from keras.callbacks import Callback
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN","NAME", "AFFILIATION"], 1)

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_counts[application_counts < 420].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df['CLASSIFICATION'].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C3700        1
C4200        1
C1283        1
C4120        1
C1236        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_count_great_one = class_counts[class_counts > 1]
class_count_great_one

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1278       10
C1238       10
C1237        9
C1235        9
C7210        7
C2400        6
C4100        6
C1720        6
C1257        5
C1600        5
C0           3
C2710        3
C1260        3
C1267        2
C1256        2
C1234        2
C1246        2
C3200        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_counts[class_counts < 1500].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_dummies = pd.get_dummies(application_df)
application_df_dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [10]:
# Split our preprocessed data into our features and target arrays
X = application_df_dummies.drop(['IS_SUCCESSFUL'], axis = 'columns').values
y = application_df_dummies['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input_layer = len(X_train[0])
hidden_node_layers1 = 100
hidden_node_layers2 = 50
hidden_node_layers3 = 1

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_node_layers1, input_dim = num_input_layer, activation = 'linear'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_node_layers2, activation = 'relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units = hidden_node_layers3, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 100)               3800      
                                                                 
 dense_4 (Dense)             (None, 50)                5050      
                                                                 
 dense_5 (Dense)             (None, 1)                 51        
                                                                 
Total params: 8,901
Trainable params: 8,901
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Checkpoint file creation
os.makedirs('checkpoints/', exist_ok = True)
checkpoint_dir = r'checkpoints/Checkpoints.{epoch:02d}.hdf5'

In [20]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

checkpoint = ModelCheckpoint(filepath = checkpoint_dir, monitor = 'accuracy', verbose = 1, save_weights_only = True,
                            mode = 'auto', period = 1)

In [21]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs = 75, callbacks = [checkpoint])

Epoch 1/75
746/804 [==========================>...] - ETA: 0s - loss: 0.6327 - accuracy: 0.6412
Epoch 1: saving model to checkpoints\Checkpoints.01.hdf5
804/804 [==============================] - 1s 693us/step - loss: 0.6325 - accuracy: 0.6417
Epoch 2/75
786/804 [============================>.] - ETA: 0s - loss: 0.6181 - accuracy: 0.6507
Epoch 2: saving model to checkpoints\Checkpoints.02.hdf5
804/804 [==============================] - 1s 714us/step - loss: 0.6191 - accuracy: 0.6505
Epoch 3/75
796/804 [============================>.] - ETA: 0s - loss: 0.6156 - accuracy: 0.6504
Epoch 3: saving model to checkpoints\Checkpoints.03.hdf5
804/804 [==============================] - 1s 706us/step - loss: 0.6157 - accuracy: 0.6504
Epoch 4/75
796/804 [============================>.] - ETA: 0s - loss: 0.6137 - accuracy: 0.6521
Epoch 4: saving model to checkpoints\Checkpoints.04.hdf5
804/804 [==============================] - 1s 708us/step - loss: 0.6134 - accuracy: 0.6524
Epoch 5/75
785/804 [====

786/804 [============================>.] - ETA: 0s - loss: 0.6066 - accuracy: 0.6579
Epoch 34: saving model to checkpoints\Checkpoints.34.hdf5
804/804 [==============================] - 1s 716us/step - loss: 0.6063 - accuracy: 0.6581
Epoch 35/75
735/804 [==========================>...] - ETA: 0s - loss: 0.6064 - accuracy: 0.6577
Epoch 35: saving model to checkpoints\Checkpoints.35.hdf5
804/804 [==============================] - 1s 693us/step - loss: 0.6065 - accuracy: 0.6570
Epoch 36/75
738/804 [==========================>...] - ETA: 0s - loss: 0.6066 - accuracy: 0.6598
Epoch 36: saving model to checkpoints\Checkpoints.36.hdf5
804/804 [==============================] - 1s 698us/step - loss: 0.6061 - accuracy: 0.6597
Epoch 37/75
777/804 [===========================>..] - ETA: 0s - loss: 0.6054 - accuracy: 0.6560
Epoch 37: saving model to checkpoints\Checkpoints.37.hdf5
804/804 [==============================] - 1s 724us/step - loss: 0.6062 - accuracy: 0.6549
Epoch 38/75
736/804 [=======

761/804 [===========================>..] - ETA: 0s - loss: 0.6038 - accuracy: 0.6583
Epoch 67: saving model to checkpoints\Checkpoints.67.hdf5
804/804 [==============================] - 1s 737us/step - loss: 0.6045 - accuracy: 0.6576
Epoch 68/75
730/804 [==========================>...] - ETA: 0s - loss: 0.6038 - accuracy: 0.6583
Epoch 68: saving model to checkpoints\Checkpoints.68.hdf5
804/804 [==============================] - 1s 699us/step - loss: 0.6042 - accuracy: 0.6588
Epoch 69/75
745/804 [==========================>...] - ETA: 0s - loss: 0.6052 - accuracy: 0.6569
Epoch 69: saving model to checkpoints\Checkpoints.69.hdf5
804/804 [==============================] - 1s 688us/step - loss: 0.6045 - accuracy: 0.6584
Epoch 70/75
740/804 [==========================>...] - ETA: 0s - loss: 0.6054 - accuracy: 0.6566
Epoch 70: saving model to checkpoints\Checkpoints.70.hdf5
804/804 [==============================] - 1s 694us/step - loss: 0.6044 - accuracy: 0.6571
Epoch 71/75
787/804 [=======

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6161 - accuracy: 0.6554 - 207ms/epoch - 773us/step
Loss: 0.6160773634910583, Accuracy: 0.6553936004638672


In [22]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization2.h5')